<a href="https://colab.research.google.com/github/AryaPratamaPutra-10/Analys_Data_Lele/blob/main/chatbot%20lele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install tensorflow numpy pandas nltk


In [20]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Memuat dataset
# Specifying the delimiter using the 'sep' argument
data = pd.read_csv('https://raw.githubusercontent.com/AryaPratamaPutra-10/Analys_Data_Lele/b246d739eee8958854863ec5e0946d3c21475d1b/Dataset_QNA_Lele.csv', sep=';') # Change the delimiter here if necessary (e.g., sep='\t' for tab)


# Tokenisasi dan pembersihan
nltk.download('punkt')
def clean_text(text):
    tokens = word_tokenize(text.lower())
    return ' '.join([t for t in tokens if t.isalnum()])

data['Pertanyaan'] = data['Pertanyaan'].apply(clean_text)
data['jawaban'] = data['jawaban'].apply(clean_text)

# Membuat tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Pertanyaan'])

# Mengonversi teks menjadi urutan angka
sequences = tokenizer.texts_to_sequences(data['Pertanyaan'])
max_len = max([len(seq) for seq in sequences])
sequences_padded = pad_sequences(sequences, maxlen=max_len, padding='post')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
display(data)

,Pertanyaan,jawaban
0,halo,hi penikmat kolam lele
1,apa makanan terbaik untuk lele,lele biasanya diberi makan pelet cacing dan ik...
2,berapa kali sehari lele harus diberi makan,lele sebaiknya diberi makan kali sehari untuk ...
3,berapa lama lele bisa dipanen,lele bisa dipanen setelah bulan tergantung dar...
4,apa saja jenis penyakit lele,penyakit umum pada lele termasuk bintik putih ...
5,bagaimana cara menjaga kualitas air kolam lele,kualitas air dapat dijaga dengan aerasi dan pe...


In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder

# ... (rest of your code for loading and preprocessing data) ...

# Parameter model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
lstm_units = 64

# Membuat model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    LSTM(lstm_units, return_sequences=False),
    Dense(len(label_encoder.classes_), activation='softmax') # Output layer size should match unique labels
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode target labels numerically using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data['jawaban'])

# Melatih model
# Use the encoded labels (y_encoded) instead of raw strings
model.fit(sequences_padded, y_encoded, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 1.8039 - accuracy: 0.0000e+00 - val_loss: 1.7983 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 43ms/step - loss: 1.7869 - accuracy: 0.5000 - val_loss: 1.8075 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 41ms/step - loss: 1.7700 - accuracy: 0.7500 - val_loss: 1.8169 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 42ms/step - loss: 1.7532 - accuracy: 0.7500 - val_loss: 1.8266 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 43ms/step - loss: 1.7362 - accuracy: 0.7500 - val_loss: 1.8365 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 42ms/step - loss: 1.7189 - accuracy: 0.7500 - val_loss: 1.8469 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 45ms/step - loss: 1.7010 - accuracy: 0.7500 - val_loss: 1.8579 - val_accuracy: 0.0000e+

In [26]:
def respond_to_question(question):
    question_seq = tokenizer.texts_to_sequences([clean_text(question)])
    question_padded = pad_sequences(question_seq, maxlen=max_len, padding='post')
    pred = model.predict(question_padded)
    response_index = np.argmax(pred)
    response = tokenizer.index_word[response_index] if response_index in tokenizer.index_word else 'Maaf, saya tidak mengerti.'
    return response

# Contoh penggunaan
user_input = "halo"
response = respond_to_question(user_input)
print("Chatbot:", response)


1/1 [==============================] - 0s 25ms/step
Chatbot: Maaf, saya tidak mengerti.


In [36]:
def predict_class(sentence):
    # 1. Bersihkan pertanyaan input (Asumsikan Anda memiliki fungsi clean_text)
    cleaned_sentence = clean_text(sentence)

    # 2. Tokenisasi kalimat menggunakan tokenizer yang sama yang digunakan selama pelatihan
    sentence_sequence = tokenizer.texts_to_sequences([cleaned_sentence])

    # 3. Padding sequence agar sesuai dengan max_len yang digunakan selama pelatihan
    sentence_padded = pad_sequences(sentence_sequence, maxlen=max_len, padding='post')

    # 4. Sekarang prediksi menggunakan sequence yang telah di-padding
    res = model.predict(sentence_padded)[0]


    # 3. Make the prediction (get probabilities for each class)
    # Use sentence_padded instead of question_padded
    prediction_probs = model.predict(sentence_padded)[0]

    # 4. Get the index of the highest probability
    predicted_class_index = np.argmax(prediction_probs)

    # 5. Get the actual answer using the label encoder
    predicted_answer = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_answer

# Example usage
user_question = "halo"
predicted_answer = predict_answer(user_question)  # Call predict_answer instead of predict_class
print(f"Pertanyaan: {user_question}")
print(f"Jawaban yang diprediksi: {predicted_answer}")

1/1 [==============================] - 0s 24ms/step
Pertanyaan: halo
Jawaban yang diprediksi: hi penikmat kolam lele
